In [1]:
from sklearn.datasets import fetch_openml

# Load the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)

# Display the keys to understand what metadata is available
print("Keys in the dataset:", mnist.keys())

Keys in the dataset: dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml

# Load the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X = mnist['data'].to_numpy()  # Convert to NumPy array
y = mnist['target'].to_numpy()

# Function to display an image with smaller size
def show_digit(index):
    image = X[index].reshape(28, 28)
    label = y[index]
    
    plt.figure(figsize=(3, 3))  # Set figure size to 3x3 inches
    plt.imshow(image, cmap="gray")
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

# Check the type of X and y
print(f"Type of X: {type(X)}")
print(f"Type of y: {type(y)}")

# Check a few other details
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Type of X: <class 'numpy.ndarray'>
Type of y: <class 'numpy.ndarray'>
Shape of X: (70000, 784)
Shape of y: (70000,)


In [3]:
from sklearn.model_selection import train_test_split

# Convert target labels to integers (currently they are strings)
y = y.astype(int)

# Split the data: first into train+validation (60,000) and test (10,000)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)

# Now split train+validation into training (50,000) and validation (10,000)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

# Display sizes of the splits
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 50000
Validation set size: 10000
Test set size: 10000


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Step 1: Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Step 2: Train the classifier on the training set
rf_clf.fit(X_train, y_train)

# Step 3: Predict on the validation set
y_val_pred = rf_clf.predict(X_val)

# Step 4: Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of Random Forest: {val_accuracy:.4f}")

Validation Accuracy of Random Forest: 0.9692


In [5]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Step 1: Initialize the Extra-Trees classifier
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

# Step 2: Train the classifier on the training set
et_clf.fit(X_train, y_train)

# Step 3: Predict on the validation set
y_val_pred = et_clf.predict(X_val)

# Step 4: Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of Extra-Trees Classifier: {val_accuracy:.4f}")

Validation Accuracy of Extra-Trees Classifier: 0.9709


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Step 1: Initialize the SVM classifier with an RBF kernel
svm_clf = SVC(kernel='rbf', gamma='scale', random_state=42)

# Step 2: Train the classifier on the training set
svm_clf.fit(X_train, y_train)

# Step 3: Predict on the validation set
y_val_pred = svm_clf.predict(X_val)

# Step 4: Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of SVM Classifier: {val_accuracy:.4f}")

Validation Accuracy of SVM Classifier: 0.9788


In [7]:
from sklearn.ensemble import VotingClassifier

# Step 1: Initialize the voting classifier with hard voting
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf), ('svm', svm_clf)],
    voting='hard'
)

# Step 2: Train the ensemble on the training set
voting_clf.fit(X_train, y_train)

# Step 3: Predict on the validation set
y_val_pred = voting_clf.predict(X_val)

# Step 4: Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of Voting Classifier (Hard Voting): {val_accuracy:.4f}")

Validation Accuracy of Voting Classifier (Hard Voting): 0.9740


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

# Define the refined parameter grid
param_distributions = {
    'n_estimators': [200, 300, 400, 500],  # Focus on higher numbers of trees
    'max_depth': [20, 30, None],           # Removed lower depths for deeper trees
    'min_samples_split': [2, 5, 10],       # Retained common split values
    'max_features': ['sqrt', 'log2']       # Removed 'None' to prevent long runs
}

# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    rf_clf, param_distributions, 
    n_iter=10,  # Number of parameter settings to try
    scoring='accuracy',  # Use accuracy as the evaluation metric
    cv=3,  # Increased to 3-fold cross-validation for more reliable evaluation
    verbose=1,  # Moderate verbosity to track progress without excessive output
    random_state=42,
    n_jobs=-2  # Use all but one core
)

# Run the random search on the training data
random_search.fit(X_train, y_train)

# Get the best parameters and evaluate on the validation set
best_rf_clf = random_search.best_estimator_
y_val_pred = best_rf_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Best parameters for Random Forest:", random_search.best_params_)
print(f"Validation Accuracy of Tuned Random Forest: {val_accuracy:.4f}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters for Random Forest: {'n_estimators': 400, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': None}
Validation Accuracy of Tuned Random Forest: 0.9709


In [10]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

# Define the parameter grid
param_distributions = {
    'n_estimators': [200, 300, 400, 500],  # Focus on higher values for n_estimators
    'max_depth': [20, 30, None],           # Limit depth or allow fully grown trees
    'min_samples_split': [2, 5, 10],       # Common split thresholds
    'min_samples_leaf': [1, 2, 4],         # Control the minimum leaf size
    'max_features': ['sqrt', 'log2']       # Restrict to common feature selection strategies
}

# Initialize the Extra-Trees classifier
et_clf = ExtraTreesClassifier(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    et_clf, param_distributions, 
    n_iter=10,  # Number of parameter settings to try
    scoring='accuracy',  # Use accuracy as the evaluation metric
    cv=3,  # 3-fold cross-validation
    verbose=1,  # Moderate verbosity
    random_state=42,
    n_jobs=9  # Use all but one core
)

# Run the random search on the training data
random_search.fit(X_train, y_train)

# Get the best parameters and evaluate on the validation set
best_et_clf = random_search.best_estimator_
y_val_pred = best_et_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Best parameters for Extra-Trees:", random_search.best_params_)
print(f"Validation Accuracy of Tuned Extra-Trees: {val_accuracy:.4f}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=20, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=19.3min


/opt/anaconda3/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=20, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=19.2min
Best parameters for Extra-Trees: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30}
Validation Accuracy of Tuned Extra-Trees: 0.9736


In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the SVM classifier with the correct parameters
svm_clf = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)

# Train the classifier on the training set
svm_clf.fit(X_train, y_train)

# Predict on the validation set
y_val_pred = svm_clf.predict(X_val)

# Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of SVM Classifier with Correct Parameters: {val_accuracy:.4f}")

Validation Accuracy of SVM Classifier with Correct Parameters: 0.9852


In [19]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Use the SVM model with best parameters and probability enabled
svm_clf = SVC(kernel='rbf', C=10, gamma='scale', probability=True, random_state=42)

# Step 1: Initialize the soft voting classifier
voting_clf_soft = VotingClassifier(
    estimators=[
        ('rf', best_rf_clf),
        ('et', best_et_clf),
        ('svm', svm_clf)
    ],
    voting='soft'  # Use soft voting to average predicted probabilities
)

# Step 2: Train the ensemble on the training set
voting_clf_soft.fit(X_train, y_train)

# Step 3: Predict on the validation set
y_val_pred = voting_clf_soft.predict(X_val)

# Step 4: Evaluate the accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy of Soft Voting Classifier: {val_accuracy:.4f}")

Validation Accuracy of Soft Voting Classifier: 0.9841


In [20]:
# Predict on the test set
y_test_pred = voting_clf_soft.predict(X_test)

# Evaluate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy of Soft Voting Classifier: {test_accuracy:.4f}")

Test Accuracy of Soft Voting Classifier: 0.9805


In [ ]:
# ---------------------------------------------
# Project Summary: Ensemble Learning with MNIST
# ---------------------------------------------

# 1. Problem Statement:
#    - Classify digits (0-9) from the MNIST dataset using ensemble methods.
#    - Split data into training (50k), validation (10k), and test (10k) sets.
#    - Train hyperparameter-tuned Random Forest, Extra-Trees, and SVM classifiers,
#      and combine them using voting ensembles.

# 2. Individual Classifier Results (after hyperparameter tuning):
#    - Random Forest: Validation Accuracy ~ 97.09%
#    - Extra-Trees: Validation Accuracy ~ 97.36%
#    - SVM (RBF kernel): Validation Accuracy ~ 98.52%

# 3. Ensemble Approach:
#    a) Hard Voting:
#       - Combined classifiers using hard voting.
#       - Validation Accuracy: ~ 97.40%
#    b) Soft Voting:
#       - Combined classifiers using soft voting (averaging probabilities).
#       - Validation Accuracy: ~ 98.41%
#       - Test Accuracy: ~ 98.05%

# 4. Conclusion:
#    - The soft voting ensemble achieved the best performance,
#      with a final test accuracy of 98.05%.
#    - Combining hyperparameter-tuned models using soft voting improved
#      overall robustness compared to individual classifiers and hard voting.